In [1]:
#variables:
#primer dilutions:
stkprm = 100 #concentration of the stock primer you are adding
stkvol = 1 #the volume of stock primer you are adding
dilprm = 2.5 #this is the concentration in uM that you want your working dilution to be

#pcr reaction
# need to get this from the df##Numprimers = 4 #this is how many primers go in each pcr reaction.
primerconcentration = 0.1 #this is the concentration you want each primer to be in the pcr reaction
pcrvol = 25 #this is the total volume of your pcr reaction 
templatengs = .5 #this is the concentration of template you want in your pcr rxn in ng/uL

#template dilutions tells you what the temps need to be diluted to initially so that you can just add 1 uL of template to the pcr:
#need to fill in stock template values further down the script
diltemp = (templatengs)*(pcrvol)/1

total_volume = 25
Q5 = total_volume - (0.5*(total_volume)) #How much Q5 to add
DPNI = 1 #How much DPNI to add
DPwater = 19
cutsmart = 5

Date = '20210908'

#first import information from the j5 spreadsheet in order to perform appropriate steps
#import feather
#import pyarrow.feather as ft
import pandas
import numpy as np
import os

In [2]:

os.chdir("C:/Users/jonbr/Documents/GitHub/opentrons/Cloning/20210915_IVA")
os.getcwd()
oligos = pandas.read_csv('oligo.csv')
oligos


oligos['ID Number'] = oligos['ID Number'].astype(int)
oligos

if len(oligos.columns) < 9:
    oligos['well'] = ''
    oligos['stock primer concentration'] = ''
    oligos['volume of stock primer to add'] = ''
    oligos['concentration of diluted primer'] = ''
    oligos['volume of diluted primer'] = '' #this is a calculated value
    oligos['how much of the diluted primer is left'] = '' #also a calculated value
oligos

#custom 4x6 well plate dictionary. hardcoded specifically for the labware used. 
#this could easily be replace with another well specification dictionary

id2well = {}
id2well['0'] = 'A1'
id2well['1'] = 'A2'
id2well['2'] = 'A3'
id2well['3'] = 'A4'
id2well['4'] = 'A5'
id2well['5'] = 'A6'
id2well['6'] = 'B1'
id2well['7'] = 'B2'
id2well['8'] = 'B3'
id2well['9'] = 'B4'
id2well['10'] = 'B5'
id2well['11'] = 'B6'
id2well['12'] = 'C1'
id2well['13'] = 'C2'
id2well['14'] = 'C3'
id2well['15'] = 'C4'
id2well['16'] = 'C5'
id2well['17'] = 'C6'
id2well['18'] = 'D1'
id2well['19'] = 'D2'
id2well['20'] = 'D3'
id2well['21'] = 'D4'
id2well['22'] = 'D5'
id2well['23'] = 'D6'

for i, row in oligos.iterrows():
    oligos.loc[i,'well'] = id2well[str(i)] #this only works because the index matces the id number. id number is a floating value
    oligos.loc[i,'stock primer concentration'] = stkprm
    oligos.loc[i,'volume of stock primer to add'] = stkvol
    oligos.loc[i,'concentration of diluted primer'] = dilprm
    
for i, row in oligos.iterrows():
    oligos.loc[i,'volume of diluted primer'] = row['stock primer concentration']*row['volume of stock primer to add']/row['concentration of diluted primer']
    
oligos

#oligos.to_csv('output_'+Date+'_oligo_IVA.csv')


In [3]:
os.chdir("C:/Users/jonbr/Documents/GitHub/opentrons/Cloning/20210915_IVA")
os.getcwd()
pcr = pandas.read_csv('pcr.csv')
pcr.columns = pcr.columns.str.replace("'","")
pcr

pcr[['Reaction ID Number','Forward Oligo ID Number','Reverse Oligo ID Number']] = pcr[['Reaction ID Number','Forward Oligo ID Number','Reverse Oligo ID Number']].astype(int)
pcr


,Reaction ID Number,Primary Template,Alternate Template,Forward Oligo ID Number,Forward Oligo Name,Reverse Oligo ID Number,Reverse Oligo Name,Notes,Mean Oligo Tm,Delta Oligo Tm,Mean Oligo Tm (3 Only),Delta Oligo Tm (3Only),Length,Sequence
0,0,pWL87_pGP8A_ARF1,NaN,0,oWL00273_(piece-1)_forward,1,oWL00201-mas00002_(ARF19_0-010)_reverse,PCR,62.5915,21.131,59.4990,14.946,3159,TTGCTCTCGGTCAAGCTTTTAAAGAGGCCCTAGGGGCCGTGCGTGG...
1,1,pWL87_pGP8A_ARF1,NaN,2,oWL00200-mas00003_(ARF19_051-end)_forward,3,oWL00274_(piece-2)_reverse,PCR,71.7340,0.614,65.4245,0.271,3527,CCTTTTTACATATGTCCTGCACAGAGTTCGATGTTCCCTGGTTTAA...
2,2,pWL87_pGP8A_ARF1,NaN,4,oWL00275_(piece-3)_forward,5,oWL00276_(piece-3)_reverse,PCR,72.7495,5.383,67.8095,2.419,2940,CTAATGAGTGAGGTAACTCACATTAATTGCGTTGCGCTCACTGCCC...
3,3,pWL87_pGP8A_ARF1,NaN,0,oWL00273_(piece-1)_forward,6,oWL00196-mas00007_(ARF19_0-200)_reverse,PCR,65.2345,15.845,62.1420,9.660,3729,TTGCTCTCGGTCAAGCTTTTAAAGAGGCCCTAGGGGCCGTGCGTGG...
4,4,pWL87_pGP8A_ARF1,NaN,7,oWL00277_(alt_piece-2)_forward,3,oWL00274_(piece-2)_reverse,PCR,72.2840,0.486,62.7575,5.063,2957,CCTACTGGATTCTCTCAGCAGCAGCAACTGGAAATGCATAACAGTA...


In [4]:
#here we create an object with each unique template from the
templates = pcr["Primary Template"]
unique_templates = templates.drop_duplicates(keep = 'first', inplace = False)
unique_templates

df = pandas.DataFrame(unique_templates)

df = df.reset_index()
df = df.drop('index', 1)

df['Template Concentration'] = ''
df


,Primary Template,Template Concentration
0,pWL87_pGP8A_ARF1,


In [5]:
#enter template concentrations here
df['Template Concentration'] = [455] #all you have to do is input the template concentrations in the right order

df

startnum = len(oligos['well'])

for i, row in df.iterrows():
    df.loc[i,'template_well'] = id2well[str(startnum+i)]
    


for i, row in df.iterrows():
    df.loc[i,'amount of template to add'] = 1


for i, row in df.iterrows():
   
        df.loc[i,'concentration of template (ng/uL)'] = diltemp

df['volume of dilute template prepared'] = df['Template Concentration']*stkvol/df['concentration of template (ng/uL)']

df['water to add']= df['volume of dilute template prepared']-df['amount of template to add']


#df['volume of dilute template prepared'] = df['volume of dilute template prepared'].astype(float)
#df['concentration of template (ng/uL)'] = df['concentration of template (ng/uL)'].astype(float)
#df['amount of template to add'] = df['amount of template to add'].astype(float)

#df['water to add'] = df['water to add'].astype(int)
#df['concentration of template (ng/uL)'] = df['concentration of template (ng/uL)'].astype(int)
#df['volume of dilute template prepared'] = df['volume of dilute template prepared'].astype(int)
#df['amount of template to add'] = df['amount of template to add'].astype(int)

df['water to add'].astype(np.float32)
df['amount of template to add'].astype(np.float32)
df['concentration of template (ng/uL)'].astype(np.float32)
df['volume of dilute template prepared'].astype(np.float32)

#df['amount of template to add'] = pd.Series.astype(df['amount of template to add'], dtype=float)
#df['concentration of template (ng/uL)'] = pd.Series.astype(df['concentration of template (ng/uL)'], dtype=float)
#df['volume of dilute template prepared'] = pd.Series.astype(df['volume of dilute template prepared'], dtype=float,)
df

#df.to_csv('output_'+Date+'_templates_IVA.csv')

df.dtypes

Primary Template                       object
Template Concentration                  int64
template_well                          object
amount of template to add             float64
concentration of template (ng/uL)     float64
volume of dilute template prepared    float64
water to add                          float64
dtype: object

In [6]:
#this line of code integrates the template concentrations into the pcr df
pcr_plustemplates = pandas.merge(pcr,df,on='Primary Template')
pcr_plustemplates

if len(pcr_plustemplates.columns) == 17:

    for i, row in pcr_plustemplates.iterrows():
   
        pcr_plustemplates.loc[i,'concentration of template (ng/uL)'] = diltemp

pcr_plustemplates['volume of dilute template prepared'] = pcr_plustemplates['Template Concentration']*stkvol/pcr_plustemplates['concentration of template (ng/uL)']

pcr_plustemplates


,Reaction ID Number,Primary Template,Alternate Template,Forward Oligo ID Number,Forward Oligo Name,Reverse Oligo ID Number,Reverse Oligo Name,Notes,Mean Oligo Tm,Delta Oligo Tm,Mean Oligo Tm (3 Only),Delta Oligo Tm (3Only),Length,Sequence,Template Concentration,template_well,amount of template to add,concentration of template (ng/uL),volume of dilute template prepared,water to add
0,0,pWL87_pGP8A_ARF1,NaN,0,oWL00273_(piece-1)_forward,1,oWL00201-mas00002_(ARF19_0-010)_reverse,PCR,62.5915,21.131,59.4990,14.946,3159,TTGCTCTCGGTCAAGCTTTTAAAGAGGCCCTAGGGGCCGTGCGTGG...,455,B3,1.0,12.5,36.4,35.4
1,1,pWL87_pGP8A_ARF1,NaN,2,oWL00200-mas00003_(ARF19_051-end)_forward,3,oWL00274_(piece-2)_reverse,PCR,71.7340,0.614,65.4245,0.271,3527,CCTTTTTACATATGTCCTGCACAGAGTTCGATGTTCCCTGGTTTAA...,455,B3,1.0,12.5,36.4,35.4
2,2,pWL87_pGP8A_ARF1,NaN,4,oWL00275_(piece-3)_forward,5,oWL00276_(piece-3)_reverse,PCR,72.7495,5.383,67.8095,2.419,2940,CTAATGAGTGAGGTAACTCACATTAATTGCGTTGCGCTCACTGCCC...,455,B3,1.0,12.5,36.4,35.4
3,3,pWL87_pGP8A_ARF1,NaN,0,oWL00273_(piece-1)_forward,6,oWL00196-mas00007_(ARF19_0-200)_reverse,PCR,65.2345,15.845,62.1420,9.660,3729,TTGCTCTCGGTCAAGCTTTTAAAGAGGCCCTAGGGGCCGTGCGTGG...,455,B3,1.0,12.5,36.4,35.4
4,4,pWL87_pGP8A_ARF1,NaN,7,oWL00277_(alt_piece-2)_forward,3,oWL00274_(piece-2)_reverse,PCR,72.2840,0.486,62.7575,5.063,2957,CCTACTGGATTCTCTCAGCAGCAGCAACTGGAAATGCATAACAGTA...,455,B3,1.0,12.5,36.4,35.4


In [7]:
wellinfo = oligos[['ID Number','well']]
wellinfo

wellinfo = wellinfo.rename(columns={'ID Number':'Forward Oligo ID Number'})
pcr_plustemplates = pcr_plustemplates.merge(wellinfo, on= 'Forward Oligo ID Number')
wellinfo = wellinfo.rename(columns={'Forward Oligo ID Number':'Reverse Oligo ID Number','well':'well2'})
pcr_plustemplates = pcr_plustemplates.merge(wellinfo, on= 'Reverse Oligo ID Number')
pcr_plustemplates

#pcr_plustemplates.to_csv('output_'+Date+'_pcr_IVA.csv')

In [8]:
#read in assembly pieces as dataframe .... might not need this info
os.chdir("C:/Users/jonbr/Documents/GitHub/opentrons/Cloning/20210915_IVA")
os.getcwd()
assembly = pandas.read_csv('assembly.csv')
assembly

#assembly.to_csv('output_'+Date+'_assembly_IVA.csv')

In [9]:
os.chdir("C:/Users/jonbr/Documents/GitHub/opentrons/Cloning/20210915_IVA")
os.getcwd()
combinations = pandas.read_csv('combinations.csv')
combinations


,ID Number,Name,Assembly Method,Part(s) Bin 0,Assembly Piece ID Number Bin 0,Part(s) Bin 1,Assembly Piece ID Number Bin 1,Part(s) Bin 2,Assembly Piece ID Number Bin 2
0,0,pmas00001,SLIC/Gibson/CPEC,(piece-1),0,(piece-2),1,(piece-3),2
1,1,pmas00002,SLIC/Gibson/CPEC,(alt_piece-1),3,(alt_piece-2),4,(piece-3),2


In [10]:
combinations[['ID Number','Assembly Piece ID Number Bin 0','Assembly Piece ID Number Bin 1','Assembly Piece ID Number Bin 2']] = combinations[['ID Number','Assembly Piece ID Number Bin 0','Assembly Piece ID Number Bin 1','Assembly Piece ID Number Bin 2']].astype(int)
combinations

primerlocations = pcr_plustemplates[['Reaction ID Number','well','well2']]
primerlocations

primerlocations = primerlocations.rename(columns={'Reaction ID Number':'Assembly Piece ID Number Bin 0'})
combinations = combinations.merge(primerlocations, on= 'Assembly Piece ID Number Bin 0')

primerlocations = primerlocations.rename(columns={'Assembly Piece ID Number Bin 0':'Assembly Piece ID Number Bin 1'})
combinations = combinations.merge(primerlocations, on= 'Assembly Piece ID Number Bin 1')

primerlocations = primerlocations.rename(columns={'Assembly Piece ID Number Bin 1':'Assembly Piece ID Number Bin 2'})
combinations = combinations.merge(primerlocations, on= 'Assembly Piece ID Number Bin 2')



combinations['primer concentrations'] = oligos['concentration of diluted primer']
combinations['amount primer to add to IVA'] = pcrvol*primerconcentration/combinations['primer concentrations']

templateinformation = pcr_plustemplates[['Reaction ID Number','template_well']]

templateinformation = templateinformation.rename(columns={'Reaction ID Number':'Assembly Piece ID Number Bin 0'})
combinations = combinations.merge(templateinformation, on= 'Assembly Piece ID Number Bin 0')

templateinformation = templateinformation.rename(columns={'Assembly Piece ID Number Bin 0':'Assembly Piece ID Number Bin 1'})
combinations = combinations.merge(templateinformation, on= 'Assembly Piece ID Number Bin 1')

templateinformation = templateinformation.rename(columns={'Assembly Piece ID Number Bin 1':'Assembly Piece ID Number Bin 2'})
combinations = combinations.merge(templateinformation, on= 'Assembly Piece ID Number Bin 2')

combinations['template concentrations'] = pcr_plustemplates['concentration of template (ng/uL)']
combinations['amount templates to add'] = pcrvol*templatengs/combinations['template concentrations']

combinations

,ID Number,Name,Assembly Method,Part(s) Bin 0,Assembly Piece ID Number Bin 0,Part(s) Bin 1,Assembly Piece ID Number Bin 1,Part(s) Bin 2,Assembly Piece ID Number Bin 2,well_x,...,well2_y,well,well2,primer concentrations,amount primer to add to IVA,template_well_x,template_well_y,template_well,template concentrations,amount templates to add
0,0,pmas00001,SLIC/Gibson/CPEC,(piece-1),0,(piece-2),1,(piece-3),2,A1,...,A4,A5,A6,2.5,1.0,B3,B3,B3,12.5,1.0
1,1,pmas00002,SLIC/Gibson/CPEC,(alt_piece-1),3,(alt_piece-2),4,(piece-3),2,A1,...,A4,A5,A6,2.5,1.0,B3,B3,B3,12.5,1.0


In [11]:
filter_col = [col for col in combinations if col.startswith('well')] #here we calculate how much water to add
filter_col
primersadded = len(filter_col)

primersadded

6

In [12]:
temp_col = [col for col in combinations if col.startswith('template_')] #here we calculate how much water to add
temp_col
tempsadded = len(temp_col)


combinations['water to add'] = (pcrvol-primersadded- tempsadded - Q5)


combinations

# id2wellpcr = {}
# id2wellpcr['0'] = 'A1'
# id2wellpcr['1'] = 'A2'
# id2wellpcr['2'] = 'A3'
# id2wellpcr['3'] = 'A4'
# id2wellpcr['4'] = 'A5'
# id2wellpcr['5'] = 'A6'
# id2wellpcr['6'] = 'A7'
# id2wellpcr['7'] = 'A8'
# id2wellpcr['8'] = 'A9'
# id2wellpcr['9'] = 'A10'
# id2wellpcr['10'] = 'A11'
# id2wellpcr['11'] = 'A12'

#Have to adjust this dictionary based on which pcr well is available.
#theoretically, we need 96 if statements with a variation of the dictionary starting at each well then you could
#just imput which well to start with 
id2wellpcr = {}
id2wellpcr['0'] = 'A3'
id2wellpcr['1'] = 'A4'
id2wellpcr['2'] = 'A5'
id2wellpcr['3'] = 'A6'
id2wellpcr['4'] = 'A7'
id2wellpcr['5'] = 'A8'
id2wellpcr['6'] = 'A9'
id2wellpcr['7'] = 'A10'
id2wellpcr['8'] = 'A11'
id2wellpcr['9'] = 'A12'
id2wellpcr['10'] = 'B1'
id2wellpcr['11'] = 'B2'

combinations['pcrwell']=combinations['ID Number']
for i, row in combinations.iterrows():
    combinations.loc[i,'pcrwell'] = id2wellpcr[str(i)]
combinations

#combinations.to_csv('output_'+Date+'_combination_IVA.csv')


In [13]:
pieces = [columns for columns in combinations if columns.startswith('Assembly Piece ID Number Bin ')]
frame = combinations[pieces]
frame2 = frame.transpose()
frame2

,0,1
Assembly Piece ID Number Bin 0,0,3
Assembly Piece ID Number Bin 1,1,4
Assembly Piece ID Number Bin 2,2,2


In [14]:
pcr_info = [columns for columns in pcr_plustemplates if columns.startswith('Mean Oligo Tm (3')]
morepcr_info = [columns for columns in pcr_plustemplates if columns.startswith('Delta Oligo Tm (3')]
anotherpcr_info = [columns for columns in pcr_plustemplates if columns.startswith('Length')]
pcr_info = pcr_info + morepcr_info + anotherpcr_info
pcr_info

['Mean Oligo Tm (3 Only)', 'Delta Oligo Tm (3Only)', 'Length']

In [15]:
for column in frame2:
    listoffrags = frame2[column].to_list()
    listoffrags
    
    tablee = pcr_plustemplates[pcr_info]
    tablee = tablee.iloc[listoffrags, :]
    
    if column == 0:
        params0 = tablee.copy()
    if column == 1:
        params1 = tablee.copy()
    if column == 2:
        params2 = tablee.copy()
    if column == 3:
        params3 = tablee.copy()
    if column == 4:
        params4 = tablee.copy()
    if column == 5:
        params5 = tablee.copy()
    if column == 6:
        params6 = tablee.copy()
    if column == 7:
        params7 = tablee.copy()
    if column == 8:
        params8 = tablee.copy()
    if column == 9:
        params9 = tablee.copy()
    if column == 10:
        params10 = tablee.copy()
    if column == 11:
        params11 = tablee.copy()

        
params0

,Mean Oligo Tm (3 Only),Delta Oligo Tm (3Only),Length
0,59.4990,14.946,3159
1,62.1420,9.660,3729
2,65.4245,0.271,3527


In [16]:
params1

,Mean Oligo Tm (3 Only),Delta Oligo Tm (3Only),Length
3,62.7575,5.063,2957
4,67.8095,2.419,2940
2,65.4245,0.271,3527


In [17]:
#so looks like this script will be limited for finding pcr conditions for just 2 construct. have to update for more

Lengthparams0 = params0.nlargest(1,'Length')
Lengthparams0['Length'] = (Lengthparams0['Length']/1000)*30
lengthlist = Lengthparams0['Length'].to_list()

Lengthparams1 = params1.nlargest(1,'Length')
Lengthparams1['Length'] = (Lengthparams1['Length']/1000)*30
lengthlist1 = Lengthparams1['Length'].to_list()

#Lengthparams2 = params2.nlargest(1,'Length')
#   Lengthparams2['Length'] = (Lengthparams2['Length']/1000)*60
 #   lengthlist = Lengthparams2['Length'].to_list()
  #  print(lengthlist)

#Lengthparams3 = params0.nlargest(1,'Length')
#Lengthparams0['Length'] = (Lengthparams0['Length']/1000)*60
#lengthlist = Lengthparams0['Length'].to_list()
#print(lengthlist)

#Lengthparams0 = params0.nlargest(1,'Length')
#Lengthparams0['Length'] = (Lengthparams0['Length']/1000)*60
#lengthlist = Lengthparams0['Length'].to_list()
#print(lengthlist)

finallengthlist = lengthlist + lengthlist1
finallengthlist

[111.87, 105.81]

In [18]:
combinations['Extension_time_sec'] = finallengthlist
combinations

,ID Number,Name,Assembly Method,Part(s) Bin 0,Assembly Piece ID Number Bin 0,Part(s) Bin 1,Assembly Piece ID Number Bin 1,Part(s) Bin 2,Assembly Piece ID Number Bin 2,well_x,...,primer concentrations,amount primer to add to IVA,template_well_x,template_well_y,template_well,template concentrations,amount templates to add,water to add,pcrwell,Extension_time_sec
0,0,pmas00001,SLIC/Gibson/CPEC,(piece-1),0,(piece-2),1,(piece-3),2,A1,...,2.5,1.0,B3,B3,B3,12.5,1.0,3.5,A3,111.87
1,1,pmas00002,SLIC/Gibson/CPEC,(alt_piece-1),3,(alt_piece-2),4,(piece-3),2,A1,...,2.5,1.0,B3,B3,B3,12.5,1.0,3.5,A4,105.81


In [19]:
extens = combinations.nlargest(1,'Extension_time_sec')
extension_final = extens['Extension_time_sec']
extension_final

0    111.87
Name: Extension_time_sec, dtype: float64

In [20]:
params0

,Mean Oligo Tm (3 Only),Delta Oligo Tm (3Only),Length
0,59.4990,14.946,3159
1,62.1420,9.660,3729
2,65.4245,0.271,3527


In [21]:
params0['Upper_temp'] = params0['Mean Oligo Tm (3 Only)'] + params0['Delta Oligo Tm (3Only)']
params0['Lower_temp'] = params0['Mean Oligo Tm (3 Only)'] - params0['Delta Oligo Tm (3Only)']
params0

,Mean Oligo Tm (3 Only),Delta Oligo Tm (3Only),Length,Upper_temp,Lower_temp
0,59.4990,14.946,3159,74.4450,44.5530
1,62.1420,9.660,3729,71.8020,52.4820
2,65.4245,0.271,3527,65.6955,65.1535


In [22]:
Lowest_high = params0.nsmallest(1,'Upper_temp')
Lowest_high

,Mean Oligo Tm (3 Only),Delta Oligo Tm (3Only),Length,Upper_temp,Lower_temp
2,65.4245,0.271,3527,65.6955,65.1535


In [23]:
Highest_low = params0.nlargest(1,'Lower_temp')
LH = Lowest_high['Upper_temp']
HL=Highest_low['Lower_temp']
A = LH-HL
if A.all() > 0:
    annealing_temp = Lowest_high['Upper_temp']
if A.all() < 0:
    annealing_temp = (Lowest_high['Upper_temp']+Highest_low['Lower_temp'])/2


In [24]:
Annealing_and_extension = pandas.DataFrame({'Annealing temp': annealing_temp,
                   'extension time (seconds)': extension_final})
Annealing_and_extension = Annealing_and_extension.reset_index()
Annealing_and_extension = Annealing_and_extension.drop(columns = ['index'])

In [25]:
#Annealing_and_extension.to_csv('output_'+Date+'_Annealing_extension.csv')